In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("FraudAnalytics") \
    .enableHiveSupport() \
    .getOrCreate()


df_raw_sample = spark.read.csv("C:\\Users\\pradh\\Desktop\\archive (1)\\Synthetic_Financial_datasets_log.csv", header=True, inferSchema=True)
df_raw_sample.printSchema()
df_raw_sample.show(5, truncate=False)


root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|type    |amount  |nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|1   |PAYMENT |9839.64 |C1231006815|170136.0     |160296.36     |M1979787155|0.0           |0.0           |0      |0             |
|1   |PAY

In [3]:
from pyspark.sql.types import *

schema = StructType([
    StructField("step", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("nameOrig", StringType(), True),
    StructField("oldbalanceOrg", DoubleType(), True),
    StructField("newbalanceOrg", DoubleType(), True),
    StructField("nameDest", StringType(), True),
    StructField("oldbalanceDest", DoubleType(), True),
    StructField("newbalanceDest", DoubleType(), True),
    StructField("isFraud", IntegerType(), True),
    StructField("isFlaggedFraud", IntegerType(), True),
])

df = spark.read.csv("C:\\Users\\pradh\\Desktop\\archive (1)\\Synthetic_Financial_datasets_log.csv", header=True, schema=schema)


In [4]:
# Row count
total_rows = df.count(); print("rows:", total_rows)

# Distinct values & sanity of key columns
df.select("type").distinct().show(50)
df.groupBy("isFraud").count().orderBy("isFraud").show()
df.groupBy("isFlaggedFraud").count().orderBy("isFlaggedFraud").show()

# Null counts column 
from pyspark.sql.functions import sum as _sum
df.select([_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

# Numeric summaries
df.select("amount","oldbalanceOrg","newbalanceOrg","oldbalanceDest","newbalanceDest").summary("count","min","max","mean","stddev").show()


rows: 6362620
+--------+
|    type|
+--------+
|TRANSFER|
| CASH_IN|
|CASH_OUT|
| PAYMENT|
|   DEBIT|
+--------+

+-------+-------+
|isFraud|  count|
+-------+-------+
|      0|6354407|
|      1|   8213|
+-------+-------+

+--------------+-------+
|isFlaggedFraud|  count|
+--------------+-------+
|             0|6362604|
|             1|     16|
+--------------+-------+

+----+----+------+--------+-------------+-------------+--------+--------------+--------------+-------+--------------+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrg|nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+----+------+--------+-------------+-------------+--------+--------------+--------------+-------+--------------+
|   0|   0|     0|       0|            0|            0|       0|             0|             0|      0|             0|
+----+----+------+--------+-------------+-------------+--------+--------------+--------------+-------+--------------+

+-------+------------------+-------

In [5]:
from pyspark.sql.functions import when

# Negative checks
neg_amount = df.filter(col("amount") < 0).count()
neg_org = df.filter((col("oldbalanceOrg") < 0) | (col("newbalanceOrg") < 0)).count()
neg_dest = df.filter((col("oldbalanceDest") < 0) | (col("newbalanceDest") < 0)).count()
print("neg_amount:", neg_amount, "neg_org:", neg_org, "neg_dest:", neg_dest)

# Duplicates
dups = df.count() - df.dropDuplicates().count()
print("exact_duplicate_rows:", dups)

# Weird flags
df.groupBy("isFraud","isFlaggedFraud").count().orderBy("isFraud","isFlaggedFraud").show()


neg_amount: 0 neg_org: 0 neg_dest: 0
exact_duplicate_rows: 0
+-------+--------------+-------+
|isFraud|isFlaggedFraud|  count|
+-------+--------------+-------+
|      0|             0|6354407|
|      1|             0|   8197|
|      1|             1|     16|
+-------+--------------+-------+



In [6]:

df_clean = df.dropDuplicates()

print("After cleaning:", df_clean.count())


After cleaning: 6362620


In [7]:
# Fraud vs non-fraud 
df_clean.groupBy("isFraud").count().show()

# Amount summary
df_clean.select("amount").summary("count","min","max","mean","stddev").show()


+-------+-------+
|isFraud|  count|
+-------+-------+
|      1|   8213|
|      0|6354407|
+-------+-------+

+-------+------------------+
|summary|            amount|
+-------+------------------+
|  count|           6362620|
|    min|               0.0|
|    max|     9.244551664E7|
|   mean|179861.90354913048|
| stddev| 603858.2314629363|
+-------+------------------+



In [ ]:

df_clean.createOrReplaceTempView("transactions_clean")                                             #HIVEQL type queries 


fraud_by_type = spark.sql("""
SELECT type,
       COUNT(*) as tx_count,             
       SUM(isFraud) as fraud_count,
       SUM(amount) as total_amount,
       SUM(CASE WHEN isFraud=1 THEN amount ELSE 0 END) as fraud_amount,
       ROUND(SUM(isFraud)/COUNT(*)*100, 4) as fraud_rate_pct
FROM transactions_clean
GROUP BY type
ORDER BY fraud_rate_pct DESC
""")
fraud_by_type.show()



+--------+--------+-----------+--------------------+-------------------+--------------+
|    type|tx_count|fraud_count|        total_amount|       fraud_amount|fraud_rate_pct|
+--------+--------+-----------+--------------------+-------------------+--------------+
|TRANSFER|  532909|       4097|4.852919872631704E11|    6.06721318401E9|        0.7688|
|CASH_OUT| 2237500|       4116|3.944129952244925E11|5.989202243829999E9|         0.184|
| CASH_IN| 1399284|          0|2.363673919124594...|                0.0|           0.0|
| PAYMENT| 2151495|          0|2.809337113836992...|                0.0|           0.0|
|   DEBIT|   41432|          0|2.2719922127999997E8|                0.0|           0.0|
+--------+--------+-----------+--------------------+-------------------+--------------+



In [9]:
fraud_by_step = spark.sql("""
SELECT step,
       COUNT(*) as tx_count,
       SUM(isFraud) as fraud_count,
       SUM(amount) as total_amount,
       SUM(CASE WHEN isFraud=1 THEN amount ELSE 0 END) as fraud_amount
FROM transactions_clean
GROUP BY step
ORDER BY step
""")
fraud_by_step.show(10)   # shows first 10 steps




+----+--------+-----------+--------------------+------------------+
|step|tx_count|fraud_count|        total_amount|      fraud_amount|
+----+--------+-----------+--------------------+------------------+
|   1|    2708|         16|2.8542918115000004E8|3740247.0100000002|
|   2|    1014|          8| 8.592160401999998E7|4186592.4800000004|
|   3|     552|          4|4.3293884419999994E7|          66832.74|
|   4|     565|         10| 7.291002857000001E7|      2.64002749E7|
|   5|     665|          6|       4.554808975E7|         381841.54|
|   6|    1660|         22|1.6431055121999997E8| 974869.6799999999|
|   7|    6837|         12|      8.3293081424E8|     1.241469406E7|
|   8|   21097|         12|3.4396024073500004E9|        1589040.41|
|   9|   37628|         19| 7.008379239430001E9|     1.147663022E7|
|  10|   35991|         11| 7.124214893709999E9|        6935977.72|
+----+--------+-----------+--------------------+------------------+
only showing top 10 rows


In [10]:
top_orig = spark.sql("""
SELECT nameOrig,
       COUNT(*) as tx_count,
       SUM(isFraud) as fraud_count,
       SUM(CASE WHEN isFraud=1 THEN amount ELSE 0 END) as fraud_amount
FROM transactions_clean
GROUP BY nameOrig
ORDER BY fraud_amount DESC
LIMIT 20
""")
top_orig.show()


+-----------+--------+-----------+------------+
|   nameOrig|tx_count|fraud_count|fraud_amount|
+-----------+--------+-----------+------------+
|C1278796725|       1|          1|       1.0E7|
|C1295280435|       1|          1|       1.0E7|
|C1945606464|       1|          1|       1.0E7|
|C1677039996|       1|          1|       1.0E7|
|C1036572575|       1|          1|       1.0E7|
|C1853514800|       1|          1|       1.0E7|
|  C63538570|       1|          1|       1.0E7|
| C819618584|       1|          1|       1.0E7|
| C525906402|       1|          1|       1.0E7|
| C180127057|       1|          1|       1.0E7|
|C1049094143|       1|          1|       1.0E7|
|C1028530067|       1|          1|       1.0E7|
|C1178177971|       1|          1|       1.0E7|
|C1274141620|       1|          1|       1.0E7|
|  C19004745|       1|          1|       1.0E7|
|C1952386173|       1|          1|       1.0E7|
|  C25889412|       1|          1|       1.0E7|
|C1041060645|       1|          1|      

In [11]:
top_dest = spark.sql("""
SELECT nameDest,
       COUNT(*) as tx_count,
       SUM(isFraud) as fraud_count,
       SUM(CASE WHEN isFraud=1 THEN amount ELSE 0 END) as fraud_amount
FROM transactions_clean
GROUP BY nameDest
ORDER BY fraud_amount DESC
LIMIT 20
""")
top_dest.show()


+-----------+--------+-----------+-------------+
|   nameDest|tx_count|fraud_count| fraud_amount|
+-----------+--------+-----------+-------------+
| C668046170|       5|          2|1.016008868E7|
| C606914329|       5|          1|        1.0E7|
| C786701128|       1|          1|        1.0E7|
|C1784785490|      13|          1|        1.0E7|
| C839316822|       2|          1|        1.0E7|
| C380259496|      24|          1|        1.0E7|
|  C39886288|       5|          1|        1.0E7|
|C1089455271|      15|          1|        1.0E7|
| C428927923|       3|          1|        1.0E7|
| C879312587|      18|          1|        1.0E7|
|C1908782637|       6|          1|        1.0E7|
|C1877706055|       3|          1|        1.0E7|
| C919151595|       1|          1|        1.0E7|
| C455244250|       1|          1|        1.0E7|
|C1291939349|       1|          1|        1.0E7|
|C1161159002|       4|          1|        1.0E7|
| C743250798|       2|          1|        1.0E7|
|C1916793179|       

In [12]:
flag_vs_fraud = spark.sql("""
SELECT isFraud, isFlaggedFraud, COUNT(*) as cnt
FROM transactions_clean
GROUP BY isFraud, isFlaggedFraud
ORDER BY isFraud, isFlaggedFraud
""")
flag_vs_fraud.show()


+-------+--------------+-------+
|isFraud|isFlaggedFraud|    cnt|
+-------+--------------+-------+
|      0|             0|6354407|
|      1|             0|   8197|
|      1|             1|     16|
+-------+--------------+-------+



In [13]:
# 1. Type
fraud_by_type.coalesce(1).write.mode("overwrite").option("header", True).csv("C:/Users/pradh/Desktop/fraud_out/fraud_by_type") #use coalesce(1) to get one  Parquet file 

# 2. Step
fraud_by_step.coalesce(1).write.mode("overwrite").option("header", True).csv("C:/Users/pradh/Desktop/fraud_out/fraud_by_step")  

# 3.  Accounts
top_orig.coalesce(1).write.mode("overwrite").option("header", True).csv("C:/Users/pradh/Desktop/fraud_out/top_orig")

# 4. Top Destination Accounts
top_dest.coalesce(1).write.mode("overwrite").option("header", True).csv("C:/Users/pradh/Desktop/fraud_out/top_dest")

# 5. Fraud vs Flagged 
flag_vs_fraud.coalesce(1).write.mode("overwrite").option("header", True).csv("C:/Users/pradh/Desktop/fraud_out/flag_vs_fraud")

